In [52]:
import numpy as np
import random # 시드 고정을 위해
import os # 시드 고정을 위해
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import torch
from torch import Tensor,tensor
from torch.utils.data import DataLoader,Dataset
from torch.nn import Module,Sequential
from sklearn.model_selection import train_test_split

device= 'cuda' if torch.cuda.is_available() else 'cpu'

db = pd.read_csv(r'D:\Project\financial_marketing\Final\Zizag\\data\DB.csv', index_col=False, encoding='cp949')

db.fillna(0,inplace=True)
db['리뷰']=db['리뷰'].str.replace('\n',' ')

In [89]:
db

,중분류,브랜드,상품명,가격,리뷰수,상품평점,닉네임,리뷰,별점,날짜,...,사이즈평가,퀄리티평가,색감평가,색감,핏,재질,퀄리티,제품상태,가격.1,두께
0,후드,더블유브이프로젝트,다크니스 후드 블랙 MJHD7427,37310,-999,4.8,dh**,S인데도 굉장히 박시하네요 사진상으론 약간 빛 바랜 검정으로 나왓는데 빛때메 그렇고...,4.0,2021-03-25 00:00:00,...,생각보다 커요,보통이에요,화면과 비슷해요,0,0.0,0.0,1.0,0.0,0.0,0.0
1,후드,더블유브이프로젝트,다크니스 후드 블랙 MJHD7427,37310,-999,4.8,88**,팔길이가 조금 긴 것 같은데 신경 쓰일 정돈 아니고 딱 오버핏이에요 기모도 따뜻해요...,4.0,2021-01-19 00:00:00,...,생각보다 커요,보통이에요,화면과 비슷해요,0,-1.0,0.0,0.0,0.0,0.0,1.0
2,후드,더블유브이프로젝트,다크니스 후드 블랙 MJHD7427,37310,-999,4.8,a_**,핏은 오버핏이고 자수가 귀여워요 근데 한번 세탁+건**지 하고 입었는데 먼지 장난아...,4.0,2022-11-21 00:00:00,...,생각보다 커요,보통이에요,화면과 비슷해요,0,1.0,0.0,-1.0,0.0,0.0,0.0
3,후드,더블유브이프로젝트,다크니스 후드 블랙 MJHD7427,37310,-999,4.8,da**,키 168에 55반 / m사이즈 박시하게 입기에 좋아요 기모도 따뜻하고 모자도 커서...,4.0,2022-11-11 00:00:00,...,생각보다 커요,아주 만족해요,화면과 비슷해요,0,1.0,0.0,0.0,0.0,0.0,1.0
4,후드,더블유브이프로젝트,다크니스 후드 블랙 MJHD7427,37310,-999,4.8,rh**,사이즈는 완전 오버핏이지만 예뻐요 근데 새옷인데도 검은옷에 겉에 흰 먼지가 많이 붙...,4.0,2021-12-22 00:00:00,...,생각보다 커요,보통이에요,화면과 비슷해요,0,-1.0,0.0,0.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5140,민소매_슬리브리스,룸제이,[1+1할인/당일발송] 올데이 면 스판 크롭 끈나시,6900,1116,4.8,ni**,일반 크롭나시 재질에 배송 평범합니다 한 3일 걸린거 같아요 근데 너무 헐렁하고 쫀...,4.0,2022-01-05 00:00:00,...,생각보다 커요,보통이에요,화면과 비슷해요,0,0.0,0.0,-1.0,0.0,0.0,0.0
5141,민소매_슬리브리스,룸제이,[1+1할인/당일발송] 올데이 면 스판 크롭 끈나시,6900,1116,4.8,rl**,배송 됐을 때 입어봤는데 적당히 크롭이고 엄청 쫀쫀하진 않았던 갓 같네요 셔츠 안에...,4.0,2023-07-01 00:00:00,...,정사이즈예요,보통이에요,화면과 비슷해요,0,1.0,0.0,0.0,0.0,0.0,0.0
5142,민소매_슬리브리스,룸제이,[1+1할인/당일발송] 올데이 면 스판 크롭 끈나시,6900,1116,4.8,ks**,재질이너무부드러워서 일단만족. 길이는 키 체형에 따라 차이잇을꺼같은데 일단 저한테는...,4.0,2022-06-20 00:00:00,...,정사이즈예요,아주 만족해요,화면과 비슷해요,0,0.0,1.0,0.0,0.0,0.0,0.0
5143,민소매_슬리브리스,룸제이,[1+1할인/당일발송] 올데이 면 스판 크롭 끈나시,6900,1116,4.8,zz**,생각보다 짧은 듯한 느낌적인 느낌이 있지만 알고산거라 괜찮아여 크롭기장 상의 입을 ...,4.0,2022-12-14 00:00:00,...,정사이즈예요,보통이에요,화면과 비슷해요,0,1.0,0.0,0.0,0.0,0.0,0.0


In [90]:
len(db['브랜드'].unique())

105

In [96]:
import plotly.graph_objs as go

# 각 열의 값이 -1인 요소의 개수 구하기
count_negative= db.eq(-1).sum()[db.eq(-1).sum()>0]

# 그래프 생성
fig = go.Figure(data=[go.Bar(x=count_negative.index, y=count_negative.values)])

# 그래프 레이아웃 설정
fig.update_layout(
    title='부정 리뷰의 분포',
    xaxis_title='열 이름',
    yaxis_title='개수',
    bargap=0.1,  # 막대 사이의 간격 조정
    plot_bgcolor='rgba(0, 0, 0, 0.7)',  # 그래프 배경색 어둡게 설정
    paper_bgcolor='rgba(0, 0, 0, 0.7)',  # 전체 배경색 어둡게 설정
    font=dict(color='white')  # 글자 색상을 흰색으로 설정
)

# 그래프 출력
fig.show()


In [ ]:
torch.nn.

In [55]:
feed_color = pd.DataFrame(db[db['색감'] == -1]['리뷰'], columns=['리뷰']).reset_index(drop=True)
feed_color

,리뷰
0,사이즈랑 재질다 괜찮아요 브라운색으로 구매했는데 생각보다 쨍한 갈색인것같아여
1,생각보다 더 크네요ㅜㅜ 한 사이즈씩 작게 했어도 괜찮았을 것 같아요 근데 네이비는 ...
2,두툼하고 괜찮아요 근데 생각보다 어둡고 커요..! 후드티 모자 큰걸 좋나하는데 적...
3,핏 낙낙하고 생각보다 엄청 따뜻해서 놀랐던 제품이에용! 워낙 크치크에서 많이 사니까...
4,생각보다 퀄 괜찮은데 조금 노란끼 도는 오트밀 색이네용 근데 어깨선이나 팔통이 너무...
...,...
304,너무 흰색인 느낌이긴하고 브라부분이 더 천이 덧대져 있어서 색깔이 다른게 아쉬워요 ...
305,리뉴얼 되기 전에 사서 정말 휘뚜루 마뚜루 너무 만족하며 입다가 재구매 했어요! ...
306,사진 상 아래가 여기제품 생각보다 더 노래요… 그리고 길이는 짧고 몸통은 널널해서 ...
307,마른편인데 엄청쫀쫀해서 좀 답답한느낌이들긴해요. 색은 흰색아니고 아이보리색? 좀 누...


In [56]:
feed_color = pd.DataFrame(db[db['핏'] == -1]['리뷰'], columns=['리뷰']).reset_index(drop=True)
feed_color

,리뷰
0,팔길이가 조금 긴 것 같은데 신경 쓰일 정돈 아니고 딱 오버핏이에요 기모도 따뜻해요...
1,사이즈는 완전 오버핏이지만 예뻐요 근데 새옷인데도 검은옷에 겉에 흰 먼지가 많이 붙...
2,너무너무너무 커요 원래 후드티 크게입는거 좋아해서 라지나 엑라 입는데 허벅지 반 이...
3,저번에 다른 후드를 엠으로 시켰을때 많이커서 이번엔 에스로 시켰습니다 딱 입기 좋네...
4,프리싸이즈라 조마조마 하긴 했는데 역시나 엄청나게 크네요... 입고 학원 갔더니 친...
...,...
1375,사이즈가 생각보다 널널하진 않네용 널널한 핏을 생각했는데 그래두 그냥 가격대비 괜찮아요
1376,조금 헐렁헐렁 큰거같긴한데 셔츠안에 입으면 딱 좋은거같아요 단독으로입기엔 좀 불편할...
1377,위에가 약간 파져서 회사에는 못 입고 가요! 기장이 한 10쎈치만 짧았으면 더 예뻤...
1378,까슬거리지도 않고 시원해서 좋아요 근데 생각보다 파임은 있는편이에요ㅜ


In [8]:
filter=['NNG','MAG','EC','VA','VA+EF','VV+ETM','NNB+JKB','VCP+EC','VCP','MAG+JX','VCN']

In [9]:
from tqdm import tqdm

train_list = []

for text in tqdm(train["리뷰"]):
    prah = []  
    lst = tokenizer.pos(text)  
    for word, pos in lst:
        if pos in filter:
            prah.append(word)  
    train_list.append(prah)  

test_list = []

for text in tqdm(test["리뷰"]):
    prah = []  
    lst = tokenizer.pos(text)  
    for word, pos in lst:
        if pos in filter:
            prah.append(word)  
    test_list.append(prah) 

#X_train, X_valid, y_train, y_valid 

100%|██████████| 1029/1029 [00:00<00:00, 1956.60it/s]


In [10]:
train = pd.DataFrame({'tokens': train_list})
train['tokens']=train['tokens'].apply(lambda x:' '.join(x))

test = pd.DataFrame({'tokens': test_list})
test['tokens']=test['tokens'].apply(lambda x:' '.join(x))


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(train["tokens"])
X_train_tfidf


X_test_tfidf = vectorizer.transform(test["tokens"])
X_test_tfidf

<1029x100 sparse matrix of type '<class 'numpy.float64'>'
	with 7256 stored elements in Compressed Sparse Row format>

In [12]:
X_train_tfidf=X_train_tfidf.toarray()
X_test_tfidf=X_test_tfidf.toarray()

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train_tfidf)
scaled_X_test = scaler.transform(X_test_tfidf)

In [21]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, KFold
# cv = KFold(5, shuffle=True, random_state=42)

# model = LogisticRegression(random_state=42)

# scores_None_scaled = cross_val_score(model,X_train_tfidf,y_train,cv=cv, scoring="accuracy", n_jobs=-1)

# scores_scaled = cross_val_score(model,scaled_X_train,y_train,cv=cv, scoring="accuracy", n_jobs=-1)

# scores_lsa = cross_val_score(model,X_train_tfidf_lsa,y_train,cv=cv, scoring="accuracy", n_jobs=-1)

# scores_None_scaled.mean(),scores_scaled.mean(),scores_lsa.mean()

(0.825804834314431, 0.8260484375184325, 0.7995661739551014)

In [22]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score, KFold
# cv = KFold(5, shuffle=True, random_state=42)

# #로버스트 (0.8248324859323573, 0.8248324859323573, 0.7866894147624722)

# model = RandomForestClassifier(random_state=42)

# scores_None_scaled = cross_val_score(model,X_train_tfidf,y_train,cv=cv, scoring="accuracy", n_jobs=-1)

# scores_scaled = cross_val_score(model,scaled_X_train,y_train,cv=cv, scoring="accuracy", n_jobs=-1)

# scores_lsa = cross_val_score(model,X_train_tfidf_lsa,y_train,cv=cv, scoring="accuracy", n_jobs=-1)


# scores_None_scaled.mean(),scores_scaled.mean(),scores_lsa.mean()

(0.8248324859323573, 0.8248324859323573, 0.7866894147624722)

In [14]:
from sklearn.model_selection import cross_val_score, KFold
cv = KFold(5, shuffle=True, random_state=42)

In [15]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

catboost_model = CatBoostClassifier(random_state=42, verbose=False)

xgboost_model = XGBClassifier(random_state=42, verbosity=0)

lightgbm_model = LGBMClassifier(random_state=42, verbosity=-1)

scores_catboost = cross_val_score(catboost_model, scaled_X_train, train_target, cv=cv, scoring="accuracy", n_jobs=-1)
scores_xgboost = cross_val_score(xgboost_model, scaled_X_train, train_target, cv=cv, scoring="accuracy", n_jobs=-1)
scores_lightgbm = cross_val_score(lightgbm_model, scaled_X_train, train_target, cv=cv, scoring="accuracy", n_jobs=-1)

mean_score_catboost = scores_catboost.mean()
mean_score_xgboost = scores_xgboost.mean()
mean_score_lightgbm = scores_lightgbm.mean()

mean_score_catboost, mean_score_xgboost, mean_score_lightgbm


(0.8272629145088416, 0.8163296724038268, 0.8245885878092227)

In [19]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255)
    }

    model = CatBoostClassifier(**params, random_seed=42, verbose=False)

    score = cross_val_score(model, scaled_X_train, train_target, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# Optuna 스터디 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print('Best trial:', study.best_trial.params)


[I 2024-02-16 15:15:40,016] A new study created in memory with name: no-name-5fb04819-2834-4363-a541-c480c0f2c7d7
/tmp/ipykernel_170796/725507769.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_170796/725507769.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
[I 2024-02-16 15:16:01,299] Trial 0 finished with value: 0.8202155269025233 and parameters: {'iterations': 61, 'learning_rate': 0.02211061498567641, 'depth': 10, 'l2_leaf_reg': 0.12011176952368173, 'border_count': 100}. Best is trial 0 with value: 0.820215

KeyboardInterrupt: 

In [20]:
model=CatBoostClassifier(**{'iterations': 170, 'learning_rate': 0.07439063688296002, 'depth': 9, 'l2_leaf_reg': 0.04468287023212953, 'border_count': 229},random_state=42)
model.fit(scaled_X_train,train_target)
result = model.predict(scaled_X_train)
count = sum(1 for value in result if value == 1)
print(count)

0:	learn: 0.6498994	total: 128ms	remaining: 21.6s
1:	learn: 0.6116340	total: 183ms	remaining: 15.4s
2:	learn: 0.5853745	total: 238ms	remaining: 13.2s
3:	learn: 0.5583087	total: 293ms	remaining: 12.1s
4:	learn: 0.5333408	total: 348ms	remaining: 11.5s
5:	learn: 0.5165304	total: 403ms	remaining: 11s
6:	learn: 0.5019549	total: 471ms	remaining: 11s
7:	learn: 0.4873927	total: 539ms	remaining: 10.9s
8:	learn: 0.4775673	total: 607ms	remaining: 10.8s
9:	learn: 0.4673568	total: 674ms	remaining: 10.8s
10:	learn: 0.4583026	total: 733ms	remaining: 10.6s
11:	learn: 0.4508692	total: 788ms	remaining: 10.4s
12:	learn: 0.4423037	total: 844ms	remaining: 10.2s
13:	learn: 0.4350138	total: 899ms	remaining: 10s
14:	learn: 0.4284750	total: 954ms	remaining: 9.86s
15:	learn: 0.4223177	total: 1.01s	remaining: 9.71s
16:	learn: 0.4175258	total: 1.06s	remaining: 9.58s
17:	learn: 0.4128629	total: 1.12s	remaining: 9.45s
18:	learn: 0.4091693	total: 1.18s	remaining: 9.34s
19:	learn: 0.4051236	total: 1.23s	remaining: 9.

In [21]:
from sklearn.metrics import accuracy_score
pred=model.predict(scaled_X_test)
accuracy_score(pred,test_target )

0.8425655976676385

In [23]:
#X_train, X_valid, y_train, y_valid 
train, test, train_target, test_target = train_test_split(color_data, target, test_size=0.2, random_state=42)
test

,리뷰,target
2646,딱 정확히 말씀드릴게요 이가격에 이퀄리티 훌륭합니다 근데 제가 마른편인데도 불구하고...,0
3313,이뻐요! 질도 너무 좋고 스판도 짱짱랍니다!근데 사이즈를 미스해서ㅜㅜ 단추가 힘겹게...,0
2232,입었을 때 쫀쫀해서 좋았는데 생각보다 짧아서 놀랐어요. 바스트가 있는 편이라 가슴 ...,0
4135,일반 맨투맨처럼 약간 널널해서 셔츠나 목폴라 안에 입고 레이어드 하고 다녀요\n내부...,0
626,꺄ㅎㅎ 셔츠 너무 맘에 들어용?? 아우터처럼 걸쳐 입었을 때 널널한 핏이랑 기장도 ...,1
...,...,...
2899,화면과 같고 한번 빨았더니 조금 작아졌습니다ㅠㅠ 세탁 시 주의해야할 것 같아요. 이...,0
4989,소재가 좀 백퍼 더울 것같은 소재? ㅠㅠ 그래도 같이 산 다른 제품들에비해 진짜 좋...,0
3456,소오올직히 저렴해서 큰 기대안했는데\n옷 질이 너무 좋네요..\n누가 무료배송에 만...,0
3094,오자마자 뜯어보고 캡이 너무 딱딱한 느낌이 들어서 불편하겠다하고 입어봤는데 편해요!...,0


In [24]:
test['target']=pred
test.to_excel('색감예측.xlsx',index_label=False)

In [25]:
test['target']=test_target.to_numpy()
test.to_excel('색감정답.xlsx',index_label=False)

In [ ]:
#딥런닝

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator(X_train, specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
train_list = [ vocab(lst) for lst in X_train ]
test_list = [ vocab(lst) for lst in X_valid]

In [ ]:
token_count = [ len(lst) for lst in train_list  ]

np.mean(token_count), np.min(token_count) , np.max(token_count)

In [70]:
data.dropna(inplace=True)

In [68]:
np.array(data.index)

array([   0,    1,    2, ..., 5142, 5143, 5144])

In [80]:
multi_target=db[['색감','핏','재질','퀄리티','제품상태','가격.1','두께']].to_numpy()
data=db[['리뷰']].reset_index(drop=True)

len(data),len(multi_target)

(5145, 5145)

In [87]:
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


msss = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in msss.split(data, multi_target):
    
    legacy, new = data.iloc[train_index], data.iloc[test_index]
    legacy_target, new_target = multi_target[train_index], multi_target[test_index]

    break

In [91]:
len(legacy),len(legacy_target)

(4116, 4116)

In [93]:
from tqdm import tqdm

legacy_list = []

for text in tqdm(legacy['리뷰']):
    prah = []  
    lst = tokenizer.pos(text)  
    for word, pos in lst:
        if pos in filter:
            prah.append(word)  
    legacy_list.append(prah)  

new_list = []

for text in tqdm(new['리뷰']):
    prah = []  
    lst = tokenizer.pos(text)  
    for word, pos in lst:
        if pos in filter:
            prah.append(word)  
    new_list.append(prah)  

100%|██████████| 1029/1029 [00:00<00:00, 2015.92it/s]


In [96]:
legacy_data = pd.DataFrame({'tokens': legacy_list})
legacy_data['tokens']=legacy_data['tokens'].apply(lambda x:' '.join(x))

new_data = pd.DataFrame({'tokens': new_list})
new_data['tokens']=new_data['tokens'].apply(lambda x:' '.join(x))

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)

legacy_data = vectorizer.fit_transform(legacy_data["tokens"])
new_data = vectorizer.transform(new_data["tokens"])


In [98]:
legacy_data_tfidf=legacy_data.toarray()
new_data_tfidf=new_data.toarray()


In [100]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

legacy_data_tfidf = scaler.fit_transform(legacy_data_tfidf)
new_data_tfidf = scaler.transform(new_data_tfidf)

In [102]:
len(legacy_data_tfidf),len(new_data_tfidf)

(4116, 1029)

In [103]:
type(legacy_data_tfidf),type(new_data_tfidf)

(numpy.ndarray, numpy.ndarray)

In [105]:
type(legacy_target)

numpy.ndarray

In [128]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np

model = RandomForestClassifier(random_state=42)


multi_target_forest = MultiOutputClassifier(model, n_jobs=-1)


multi_target_forest.fit(legacy_data_tfidf, legacy_target)

pred=multi_target_forest.predict(new_data_tfidf)

from sklearn.metrics import f1_score
import numpy as np

f1_scores = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred.shape[1]):
    f1 = f1_score(new_target[:, i], pred[:, i],average='macro')
    f1_scores.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score = np.mean(f1_scores)

print("Average F1 score:", average_f1_score)

Average F1 score: 0.4812722630229635


In [126]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 초기화
logistic_model = LogisticRegression()

# MultiOutputClassifier에 로지스틱 회귀 모델을 전달하여 다중 출력 분류기 초기화
multi_target_logistic = MultiOutputClassifier(logistic_model)

# 모델 훈련
multi_target_logistic.fit(legacy_data_tfidf, legacy_target)

# 새 데이터에 대한 예측 수행
pred = multi_target_logistic.predict(new_data_tfidf)

# F1 점수 계산
f1_scores = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred.shape[1]):
    f1 = f1_score(new_target[:, i], pred[:, i], average='macro')
    f1_scores.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score = np.mean(f1_scores)

print("Average F1 score:", average_f1_score)


Average F1 score: 0.46334519759275766


In [132]:
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

# CatBoost 분류기 초기화
catboost_model = CatBoostClassifier()

# MultiOutputClassifier에 CatBoost 분류기를 전달하여 다중 출력 분류기 초기화
multi_target_catboost = MultiOutputClassifier(catboost_model)

# 모델 훈련
multi_target_catboost.fit(legacy_data_tfidf, legacy_target)

# 새 데이터에 대한 예측 수행
pred = multi_target_catboost.predict(new_data_tfidf)

Learning rate set to 0.085047
0:	learn: 1.0174256	total: 35.2ms	remaining: 35.2s
1:	learn: 0.9561166	total: 59.4ms	remaining: 29.6s
2:	learn: 0.9008452	total: 83.4ms	remaining: 27.7s
3:	learn: 0.8551397	total: 107ms	remaining: 26.7s
4:	learn: 0.8166414	total: 131ms	remaining: 26.1s
5:	learn: 0.7851225	total: 155ms	remaining: 25.7s
6:	learn: 0.7569915	total: 179ms	remaining: 25.3s
7:	learn: 0.7356132	total: 203ms	remaining: 25.2s
8:	learn: 0.7141998	total: 227ms	remaining: 25s
9:	learn: 0.6956878	total: 252ms	remaining: 24.9s
10:	learn: 0.6780734	total: 276ms	remaining: 24.8s
11:	learn: 0.6640117	total: 300ms	remaining: 24.7s
12:	learn: 0.6501916	total: 324ms	remaining: 24.6s
13:	learn: 0.6404575	total: 348ms	remaining: 24.5s
14:	learn: 0.6313158	total: 372ms	remaining: 24.4s
15:	learn: 0.6231919	total: 395ms	remaining: 24.3s
16:	learn: 0.6138454	total: 420ms	remaining: 24.3s
17:	learn: 0.6065548	total: 445ms	remaining: 24.3s
18:	learn: 0.6005471	total: 469ms	remaining: 24.2s
19:	learn:

In [146]:
pred=pred.reshape(-1, new_target.shape[-1])

In [147]:
# F1 점수 계산
f1_scores = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred.shape[1]):
    f1 = f1_score(new_target[:, i], pred[:, i], average='macro')
    f1_scores.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score = np.mean(f1_scores)

print("Average F1 score:", average_f1_score)

Average F1 score: 0.4845363420763604


In [131]:
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score

# LightGBM 분류기 초기화
lgb_model = LGBMClassifier()

# MultiOutputClassifier에 LightGBM 분류기를 전달하여 다중 출력 분류기 초기화
multi_target_lgb = MultiOutputClassifier(lgb_model)

# 모델 훈련
multi_target_lgb.fit(legacy_data_tfidf, legacy_target)

# 새 데이터에 대한 예측 수행
pred = multi_target_lgb.predict(new_data_tfidf)

# F1 점수 계산
f1_scores = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred.shape[1]):
    f1 = f1_score(new_target[:, i], pred[:, i], average='macro')
    f1_scores.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score = np.mean(f1_scores)

print("Average F1 score:", average_f1_score)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9373
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 100
[LightGBM] [Info] Start training from score -2.801176
[LightGBM] [Info] Start training from score -0.319608
[LightGBM] [Info] Start training from score -1.547271
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9373
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 100
[LightGBM] [Info] Start training from score -1.325041
[LightGBM] [Info] Start training from score -0.828207
[LightGBM] [Info] Start training from score -1.212758
[LightGBM] [Info] Auto-choosing ro

In [176]:
from skmultilearn.problem_transform import ClassifierChain
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

# LightGBM 분류기 초기화
lgb_model = LGBMClassifier()

# Classifier Chains 모델 초기화
classifier_chain_model = ClassifierChain(classifier=lgb_model)

# Classifier Chains 모델 훈련
classifier_chain_model.fit(legacy_data_tfidf, legacy_target)

# 새 데이터에 대한 예측 수행
pred_classifier_chain = classifier_chain_model.predict(new_data_tfidf)

# F1 점수 계산
f1_scores_classifier_chain = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred_classifier_chain.shape[1]):
    f1 = f1_score(new_target[:, i], pred_classifier_chain.toarray()[:, i], average='macro')
    f1_scores_classifier_chain.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score_classifier_chain = np.mean(f1_scores_classifier_chain)

print("Classifier Chain Average F1 score:", average_f1_score_classifier_chain)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002842 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9373
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 100
[LightGBM] [Info] Start training from score -2.801176
[LightGBM] [Info] Start training from score -0.319608
[LightGBM] [Info] Start training from score -1.547271
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002035 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9376
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 101
[LightGBM] [Info] Start training from score -1.325041
[LightGBM] [Info] Start training from score -0.828207
[LightGBM] [Info] Start 

In [200]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.tree import DecisionTreeClassifier

# 이진 분류를 위한 결정 트리 분류기 초기화
lgb_model = LGBMClassifier()

# Binary Relevance 모델 초기화
binary_relevance_model = BinaryRelevance(classifier=lgb_model)

# Binary Relevance 모델 훈련
binary_relevance_model.fit(legacy_data_tfidf, legacy_target)

# 새 데이터에 대한 예측 수행
pred_binary_relevance = binary_relevance_model.predict(new_data_tfidf)

# F1 점수 계산
f1_scores_binary_relevance = []

# 각 출력에 대해 F1 점수 계산하여 리스트에 추가
for i in range(pred_binary_relevance.shape[1]):
    f1 = f1_score(new_target[:, i], pred_binary_relevance.toarray()[:, i], average='macro')
    f1_scores_binary_relevance.append(f1)

# 계산된 F1 점수의 평균 계산
average_f1_score_binary_relevance = np.mean(f1_scores_binary_relevance)

print("Binary Relevance Average F1 score:", average_f1_score_binary_relevance)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002928 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9373
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 100
[LightGBM] [Info] Start training from score -2.801176
[LightGBM] [Info] Start training from score -0.319608
[LightGBM] [Info] Start training from score -1.547271
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001991 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9373
[LightGBM] [Info] Number of data points in the train set: 4116, number of used features: 100
[LightGBM] [Info] Start training from score -1.325041
[LightGBM] [Info] Start training from score -0.828207
[LightGBM] [Info] Start 